In [1]:
!pip install pyquery

You should consider upgrading via the '/Users/wxy/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from urllib.parse import urlencode
from pyquery import PyQuery as py
import requests
import pandas as pd

In [3]:
def get_information(since_id = '',uid = 0,):
    #X-Requested-With 用来标识Ajax请求，必须得有
    #Referer 用来指明请求来源  必须有
    #User-Agent  伪装浏览器，必须有
    headers = {'Referer': 'https://m.weibo.cn',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
                'X-Requested-With': 'XMLHttpRequest'}

    params = {
                'uid': uid,
                'luicode': 10000011,
                'lfid': '231093_-_selffollowed',
                'type': int(uid),
                'value': 6620842908,
                'containerid': '107603' + str(uid),
                'since_id': since_id
    }

    # urlencode() 方法将参数转化为URL的GET请求参数
    url = 'https://m.weibo.cn/api/container/getIndex?' + urlencode(params)
    response = requests.get(url,headers = headers)
    #获得返回的 json 内容，做分析
    json_text = response.json()
    #获得 since_id 为增加页面做准备
    since_id = json_text.get('data').get('cardlistInfo').get('since_id')
    return json_text,since_id

In [4]:
def parse_json(json):
    items = json.get('data').get('cards')


    for item in items:
        item = item.get('mblog')
        weibo = {}
        weibo['发表时间'] = item.get('created_at')
        #weibo['手机类型'] = item.get('source')
        weibo['转发数量'] = item.get('reposts_count')
        weibo['内容'] = py(item.get('text')).text()
        #weibo['图片链接'] = item.get('bmiddle_pic')
        weibo['点赞数'] = item.get('attitudes_count')
        weibo['评论数'] = item.get('comments_count')
        yield weibo

In [5]:

#uid 你所要爬取的微博的ID，在响应的参数列表中可以得到，图中可以找到
uid = 6620842908
#p 爬取的页数
p = 5
s = get_information(since_id = '',uid = uid)
#解析 JSON
#parse_json(s[0])
#输出解析后的内容
'''
for i in parse_json(s[0]):
    print(i)
'''
#多页爬取
data = []
for i in range(p):
    s = get_information(since_id = s[1],uid = uid)
    for i in parse_json(s[0]):
        print(i)
        data.append(i)

pd.DataFrame(data).to_csv('tnt.csv',encoding="utf_8_sig")

{'发表时间': '08-09', '转发数量': 391862, '内容': '时代少年团超话💥#时代少年团二辑《象》#\n【舞·象·之·年】第二篇章《象》\n- Time Table -\n时间往前，少年成长\n宣言仍无惧，冒险亦无畏\n而此刻，新的篇章将开启——\n@时代少年团队长-马嘉祺\n@时代少年团-丁程鑫\n@时代少年团-宋亚轩\n@时代少年团-刘耀文\n@时代少年团-张真源\n...全文', '点赞数': 362417, '评论数': 36713}
{'发表时间': '08-08', '转发数量': 9851, '内容': '时代少年团超话 少年成长ing的模样，让未来开始有了形状关于自己，关于过往，关于梦想，更多精彩请关注@博客天下', '点赞数': 173536, '评论数': 9773}
{'发表时间': '08-07', '转发数量': 25922, '内容': '时代少年团超话💥#爱豆v力量#\nTNT《回到夏天》拍摄花絮\n穿过时间\n回到夏天\n成长如期而至\n乐声延绵不绝\n而故事，也在继续书写 时代少年团的微博视频', '点赞数': 256893, '评论数': 27006}
{'发表时间': '08-06', '转发数量': 80332, '内容': '时代少年团超话 💥#爱豆v力量#\n\n夏天总会让人成长\n但夏天的故事从来没有答案 时代少年团的微博视频', '点赞数': 358327, '评论数': 48502}
{'发表时间': '08-05', '转发数量': 32014, '内容': '时代少年团超话💥#爱豆v力量#\n又来海边\n又到夏天\n海风吹过来的时候\n我们也看向彼此\n那就，一起去更远吧💥', '点赞数': 316423, '评论数': 32355}
{'发表时间': '08-04', '转发数量': 55979, '内容': '时代少年团超话💥#爱豆v力量#\nTNT《少年•平行的夏》之一 拍摄花絮\n少年人的夏\n热血远比午后阳光滚烫\n\n擦破皮肤带来细密痛感\n汗水滴落 蒸发\n伤疤减淡 消失\n\n还好记忆被留下 时代少年团的微博视频', '点赞数': 330729, '评论数': 48080}
{'发表时间': '07-31', '转发数量': 47666, '内容': '时代少年团超话💥#爱豆